In [1]:
#!/usr/bin/env python3
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop


class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (WF+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
#         with tf.device('/gpu:1'):
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
#         with tf.device('/gpu:1'):
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
#         with tf.device('/gpu:1'):
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
#         with tf.device('/gpu:1'):
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist",\
                one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows,\
                self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0, self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)


mnist_dcgan = MNIST_DCGAN()
timer = ElapsedTimer()
mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=500)
timer.elapsed_time()


Using TensorFlow backend.


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7,

38: [D loss: 6.769485, acc: 0.500000]  [A loss: 0.000015, acc: 1.000000]
39: [D loss: 0.039925, acc: 0.996094]  [A loss: 0.000399, acc: 1.000000]
40: [D loss: 0.023369, acc: 0.996094]  [A loss: 0.004925, acc: 1.000000]
41: [D loss: 0.022527, acc: 1.000000]  [A loss: 0.040002, acc: 0.996094]
42: [D loss: 0.027577, acc: 0.996094]  [A loss: 0.097318, acc: 0.964844]
43: [D loss: 0.091679, acc: 0.970703]  [A loss: 3.833979, acc: 0.066406]
44: [D loss: 2.493423, acc: 0.509766]  [A loss: 16.101875, acc: 0.000000]
45: [D loss: 2.602032, acc: 0.500000]  [A loss: 0.024326, acc: 1.000000]
46: [D loss: 0.530208, acc: 0.718750]  [A loss: 8.002487, acc: 0.000000]
47: [D loss: 0.590704, acc: 0.652344]  [A loss: 9.677189, acc: 0.000000]
48: [D loss: 0.172872, acc: 0.970703]  [A loss: 4.136106, acc: 0.000000]
49: [D loss: 0.772182, acc: 0.593750]  [A loss: 9.329585, acc: 0.000000]
50: [D loss: 0.190000, acc: 0.953125]  [A loss: 2.512659, acc: 0.031250]
51: [D loss: 0.961534, acc: 0.525391]  [A loss: 9.

150: [D loss: 0.555625, acc: 0.640625]  [A loss: 1.610025, acc: 0.000000]
151: [D loss: 0.514939, acc: 0.830078]  [A loss: 0.919972, acc: 0.136719]
152: [D loss: 0.662136, acc: 0.537109]  [A loss: 2.588324, acc: 0.000000]
153: [D loss: 0.663009, acc: 0.541016]  [A loss: 0.532625, acc: 0.886719]
154: [D loss: 0.730069, acc: 0.503906]  [A loss: 1.070947, acc: 0.011719]
155: [D loss: 0.574223, acc: 0.751953]  [A loss: 0.871162, acc: 0.140625]
156: [D loss: 0.601056, acc: 0.597656]  [A loss: 1.103942, acc: 0.011719]
157: [D loss: 0.552151, acc: 0.707031]  [A loss: 1.155707, acc: 0.011719]
158: [D loss: 0.532884, acc: 0.679688]  [A loss: 1.436136, acc: 0.000000]
159: [D loss: 0.519356, acc: 0.802734]  [A loss: 1.051268, acc: 0.046875]
160: [D loss: 0.597501, acc: 0.562500]  [A loss: 2.023158, acc: 0.000000]
161: [D loss: 0.595792, acc: 0.691406]  [A loss: 0.574433, acc: 0.789062]
162: [D loss: 0.747483, acc: 0.503906]  [A loss: 1.470109, acc: 0.000000]
163: [D loss: 0.591904, acc: 0.738281]

261: [D loss: 0.692850, acc: 0.507812]  [A loss: 1.191884, acc: 0.015625]
262: [D loss: 0.620307, acc: 0.699219]  [A loss: 0.854562, acc: 0.210938]
263: [D loss: 0.656287, acc: 0.546875]  [A loss: 1.286908, acc: 0.000000]
264: [D loss: 0.614107, acc: 0.736328]  [A loss: 0.768664, acc: 0.316406]
265: [D loss: 0.693441, acc: 0.533203]  [A loss: 1.424506, acc: 0.000000]
266: [D loss: 0.622045, acc: 0.697266]  [A loss: 0.707710, acc: 0.507812]
267: [D loss: 0.705726, acc: 0.503906]  [A loss: 1.194848, acc: 0.000000]
268: [D loss: 0.608676, acc: 0.726562]  [A loss: 0.821577, acc: 0.230469]
269: [D loss: 0.676198, acc: 0.533203]  [A loss: 1.352546, acc: 0.000000]
270: [D loss: 0.611809, acc: 0.722656]  [A loss: 0.735264, acc: 0.445312]
271: [D loss: 0.704884, acc: 0.519531]  [A loss: 1.407935, acc: 0.000000]
272: [D loss: 0.647092, acc: 0.652344]  [A loss: 0.737859, acc: 0.359375]
273: [D loss: 0.699704, acc: 0.498047]  [A loss: 1.134573, acc: 0.007812]
274: [D loss: 0.611895, acc: 0.712891]

...

1918: [D loss: 0.673916, acc: 0.589844]  [A loss: 0.915118, acc: 0.152344]
1919: [D loss: 0.682523, acc: 0.558594]  [A loss: 1.075171, acc: 0.070312]
1920: [D loss: 0.689544, acc: 0.548828]  [A loss: 0.793721, acc: 0.296875]
1921: [D loss: 0.687709, acc: 0.564453]  [A loss: 1.015759, acc: 0.097656]
1922: [D loss: 0.681408, acc: 0.578125]  [A loss: 0.768438, acc: 0.355469]
1923: [D loss: 0.695465, acc: 0.560547]  [A loss: 1.011649, acc: 0.082031]
1924: [D loss: 0.677566, acc: 0.574219]  [A loss: 0.786893, acc: 0.347656]
1925: [D loss: 0.687088, acc: 0.542969]  [A loss: 1.003292, acc: 0.070312]
1926: [D loss: 0.695677, acc: 0.544922]  [A loss: 0.840048, acc: 0.273438]
1927: [D loss: 0.683160, acc: 0.554688]  [A loss: 0.947146, acc: 0.093750]
1928: [D loss: 0.684554, acc: 0.550781]  [A loss: 0.767243, acc: 0.390625]
1929: [D loss: 0.688776, acc: 0.550781]  [A loss: 0.997488, acc: 0.097656]
1930: [D loss: 0.674344, acc: 0.546875]  [A loss: 0.790277, acc: 0.335938]
1931: [D loss: 0.701784, 

2028: [D loss: 0.680848, acc: 0.583984]  [A loss: 0.836701, acc: 0.238281]
2029: [D loss: 0.689901, acc: 0.552734]  [A loss: 0.914929, acc: 0.152344]
2030: [D loss: 0.669767, acc: 0.593750]  [A loss: 0.813922, acc: 0.269531]
2031: [D loss: 0.688380, acc: 0.591797]  [A loss: 0.958419, acc: 0.113281]
2032: [D loss: 0.686396, acc: 0.548828]  [A loss: 0.784930, acc: 0.375000]
2033: [D loss: 0.696911, acc: 0.550781]  [A loss: 0.937421, acc: 0.148438]
2034: [D loss: 0.682650, acc: 0.537109]  [A loss: 0.856251, acc: 0.214844]
2035: [D loss: 0.706367, acc: 0.537109]  [A loss: 0.959473, acc: 0.097656]
2036: [D loss: 0.671911, acc: 0.589844]  [A loss: 0.814787, acc: 0.277344]
2037: [D loss: 0.684520, acc: 0.552734]  [A loss: 0.985107, acc: 0.097656]
2038: [D loss: 0.682741, acc: 0.568359]  [A loss: 0.788791, acc: 0.339844]
2039: [D loss: 0.696777, acc: 0.537109]  [A loss: 1.023396, acc: 0.074219]
2040: [D loss: 0.666496, acc: 0.603516]  [A loss: 0.729465, acc: 0.437500]
2041: [D loss: 0.697403, 

2138: [D loss: 0.695466, acc: 0.535156]  [A loss: 0.954253, acc: 0.097656]
2139: [D loss: 0.683557, acc: 0.570312]  [A loss: 0.806529, acc: 0.308594]
2140: [D loss: 0.688632, acc: 0.539062]  [A loss: 1.017289, acc: 0.050781]
2141: [D loss: 0.674810, acc: 0.556641]  [A loss: 0.777344, acc: 0.343750]
2142: [D loss: 0.676043, acc: 0.572266]  [A loss: 0.924209, acc: 0.164062]
2143: [D loss: 0.681699, acc: 0.599609]  [A loss: 0.780453, acc: 0.375000]
2144: [D loss: 0.693947, acc: 0.542969]  [A loss: 0.851920, acc: 0.214844]
2145: [D loss: 0.695465, acc: 0.546875]  [A loss: 0.997465, acc: 0.117188]
2146: [D loss: 0.679723, acc: 0.570312]  [A loss: 0.700288, acc: 0.535156]
2147: [D loss: 0.722314, acc: 0.503906]  [A loss: 1.014814, acc: 0.070312]
2148: [D loss: 0.682932, acc: 0.570312]  [A loss: 0.764110, acc: 0.351562]
2149: [D loss: 0.688553, acc: 0.576172]  [A loss: 0.902104, acc: 0.128906]
2150: [D loss: 0.695307, acc: 0.496094]  [A loss: 0.784372, acc: 0.343750]
2151: [D loss: 0.705272, 

2248: [D loss: 0.675444, acc: 0.589844]  [A loss: 0.842280, acc: 0.218750]
2249: [D loss: 0.679892, acc: 0.546875]  [A loss: 0.892090, acc: 0.195312]
2250: [D loss: 0.680325, acc: 0.552734]  [A loss: 0.863493, acc: 0.199219]
2251: [D loss: 0.691494, acc: 0.529297]  [A loss: 0.834850, acc: 0.203125]
2252: [D loss: 0.685713, acc: 0.544922]  [A loss: 0.891213, acc: 0.171875]
2253: [D loss: 0.678677, acc: 0.560547]  [A loss: 0.764675, acc: 0.347656]
2254: [D loss: 0.689819, acc: 0.568359]  [A loss: 0.862345, acc: 0.222656]
2255: [D loss: 0.673603, acc: 0.589844]  [A loss: 0.831793, acc: 0.253906]
2256: [D loss: 0.680282, acc: 0.576172]  [A loss: 0.892747, acc: 0.167969]
2257: [D loss: 0.686023, acc: 0.556641]  [A loss: 0.810336, acc: 0.281250]
2258: [D loss: 0.700601, acc: 0.552734]  [A loss: 0.888649, acc: 0.160156]
2259: [D loss: 0.676869, acc: 0.570312]  [A loss: 0.848757, acc: 0.218750]
2260: [D loss: 0.679662, acc: 0.550781]  [A loss: 0.914767, acc: 0.160156]
2261: [D loss: 0.690130, 

KeyboardInterrupt: 